# Food Recommendations

Below is the output of a recommender system for healthy foods.  

In [1]:
# Imports and settings
import pandas as pd
from Recommender_system.Recommender import UserRatings, collabFilteringModel, getUserFavFoods, getMealRec, format_meal_rec

pd.options.display.max_colwidth = 500

In [2]:
# Load dataset
user_food_table = pd.read_csv("Recommender_system/Dataset.csv")

### Set User and Meal Kind

Here, set the user ID and kind of meal (`'breakfast'`, `'lunch'`, `'dinner'`, or `''` for all meals)

In [3]:
user_id = 1821
meal_kind = 'dinner'

## 1. Summary of User's Meals

Below are the food items in the user's food log for this meal type

In [4]:
user_ratings = UserRatings(user_food_table, meal_kind)
user_food_list, meal_summary = user_ratings.mealSummary(user_id)
meal_summary

,list of foods
meal_id,
2161,"baked ziti, ricotta cheese, mixed greens, dressing, baked ziti, ricotta cheese, mixed greens, dressing"
2179,"carrots, broccoli, chicken legs, broccoli"
2185,"chicken, salad, lettuce, tomatoes, avocado, pico de gallo, sauce"
2198,"yellow rice, pasteles, gandules, pasteles, platano"
2214,"baked ziti, salad, tangerine, baked ziti, salad, tangerine"
2233,"roast beef hero, roast beef hero, pepper jack cheese, mayo"
2286,"rice and beans, pork chop, plantain chips, white rice, red beans, pork chop, plantain chips"
2294,"liverwurst, gouda, sandwich, liverwurst, hero"
2302,"rice and beans, mixed vegetables, pork chops, white rice, red beans, pumpkin, vegetables, pork chops"


## 2. Recommending food items

In [5]:
# Run and fit the food recommendation model
food_rec_model = collabFilteringModel(d=8, sigmasq=0.6, lambd=1, nUsers=user_ratings.nUsers, nFoods=user_ratings.nFoods)
food_rec_model.fit(user_ratings.ratings_train.values,seed_u=3,seed_v=5) #set the seed to get reproducible results

collabFilteringModel(d=8, lambd=1, nFoods=558, nUsers=105, sigmasq=0.6)

In [6]:
# Get favorite and least favorite foods
fav_foods = getUserFavFoods(user_id, user_ratings, food_rec_model)
fav_foods = fav_foods[~fav_foods.food.isin(user_food_list)]

In [7]:
top_fav_foods = fav_foods.iloc[0:10,].food
top_fav_foods = top_fav_foods.sample(5, random_state=123)

In [8]:
least_fav_foods = fav_foods.iloc[-10:-1,].food
least_fav_foods = least_fav_foods.sample(5, random_state=123)

Here are 10 food items - based on what the user has eaten before, please rate the extent to which you think the user would or would not like each food item on a scale from 1, the user would not like this food item at all, to 7, the user would like this food item a lot.

In [15]:
test_food_list = pd.DataFrame({'label': 'fav', 'food': top_fav_foods})
test_food_list = test_food_list.append(pd.DataFrame({'label': 'least_fav', 'food': least_fav_foods})).sample(frac=1).reset_index()
test_food_list['rating'] = '1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)'
test_food_list[['food','rating']]

,food,rating
0,bagel,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
1,orange,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
2,avacado,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
3,veggies,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
4,protein bar,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
5,daikon,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
6,wine,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
7,fig,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
8,franks,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)
9,beer,1 (would not like at all) - 2 - 3 - 4 - 5 - 6 - 7 (would like a lot)


## 3. Recommending complete meals

In [10]:
association_rules = user_ratings.calculateAssociationRules(0.005)

In [11]:
seed_food = 'beans'

In [12]:
meal_rec = getMealRec(user_id, user_ratings=user_ratings, model=food_rec_model, seed_food=seed_food)
suggestion = meal_rec.iloc[0,:]

In [13]:
rec = format_meal_rec(suggestion, seed_food)
rec

'To go with 1 serving of beans, try 0.5 serving of tomato and rice'

How would you rate this recommendation on a scale from 1 to 7?

`1 - 2 - 3 - 4 - 5 - 6 - 7`

In [14]:
seed_food = 'chicken'
meal_rec = getMealRec(user_id, user_ratings=user_ratings, model=food_rec_model, seed_food=seed_food)
suggestion = meal_rec.iloc[0,:]
rec = format_meal_rec(suggestion, seed_food)
rec

'To go with 2 servings of chicken, try 0.5 serving of rice'

How would you rate this recommendation on a scale from 1 to 7?

`1 - 2 - 3 - 4 - 5 - 6 - 7`